In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals


!pip install -q -U tensorflow>=2
import tensorflow as tf



try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import numpy as np
import os
import time


ERROR: tensorboard 2.0.1 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.7.1 which is incompatible.
TensorFlow is already loaded. Please restart the runtime to change versions.


Read in the text

In [0]:
corpus = open("css.txt", "rb").read().decode(encoding='utf-8')

In [0]:
print ('Length of text: {} characters'.format(len(corpus)))

Length of text: 3419905 characters


In [0]:
print(corpus[:250])

/*------------------style-----------------*/

/*
Template Name: Apex App	
Author: MarkUps
Author URI: http://www.markups.io/
Version: 1.0
*/


/* Table of Content
#BASIC TYPOGRAPHY
#HEADER	
#MENU 
#F


In [0]:
# The unique characters in the file
vocab = sorted(set(corpus))
print ('{} unique characters'.format(len(vocab)))

99 unique characters


Vectorize text- map all unique characters to an integer

In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in corpus])

In [0]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\t':   0,
  '\n':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '#' :   5,
  '$' :   6,
  '%' :   7,
  '&' :   8,
  "'" :   9,
  '(' :  10,
  ')' :  11,
  '*' :  12,
  '+' :  13,
  ',' :  14,
  '-' :  15,
  '.' :  16,
  '/' :  17,
  '0' :  18,
  '1' :  19,
  ...
}


In [0]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(corpus[:25]), text_as_int[:25]))

'/*------------------style' ---- characters mapped to int ---- > [17 12 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 84 85 90 77
 70]


In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 50
examples_per_epoch = len(corpus)//(seq_length+1)

In [0]:
#convert stream of text into character indices
# Create training examples / targets

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

/
*
-
-
-


Divide characters into batch size of specific lenght

In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'/*------------------style-----------------*/\n\n/*\nTe'
'mplate Name: Apex App\t\nAuthor: MarkUps\nAuthor URI: '
'http://www.markups.io/\nVersion: 1.0\n*/\n\n\n/* Table o'
'f Content\n========================================='
'=========\n#BASIC TYPOGRAPHY\n#HEADER\t\n#MENU \n#FEATUR'


For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [0]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '/*------------------style-----------------*/\n\n/*\nT'
Target data: '*------------------style-----------------*/\n\n/*\nTe'


The input of the RNN will start from '<' and predict '!' in the next time step till the final predicted output is 't' the last character in the sequence before it moves to the next sequence.

In [0]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 17 ('/')
  expected output: 12 ('*')
Step    1
  input: 12 ('*')
  expected output: 15 ('-')
Step    2
  input: 15 ('-')
  expected output: 15 ('-')
Step    3
  input: 15 ('-')
  expected output: 15 ('-')
Step    4
  input: 15 ('-')
  expected output: 15 ('-')


In [0]:
# Batch size
BATCH_SIZE = 98

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((98, 50), (98, 50)), types: (tf.int64, tf.int64)>

In [0]:
#Batch size is usually the size of your vocabulary because you will want to maintain a reasonable proportional embedding distance between your vocab



# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(98, 50, 99) # (batch_size, sequence_length, vocab_size)


In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (98, None, 256)           25344     
_________________________________________________________________
gru (GRU)                    (98, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (98, None, 99)            101475    
Total params: 4,065,123
Trainable params: 4,065,123
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop.


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([69, 63, 20, 50,  0, 70, 52, 95,  8, 36, 41, 70, 79, 76, 46, 80, 37,
       58, 12, 97, 32, 67, 29, 26, 36, 85, 97, 79, 30, 39, 81, 54, 57, 79,
       87, 66, 88, 35, 46, 83,  1,  1, 70, 33, 90, 85, 43, 38, 92, 44])

Checking the sample text generated by this untrained model

In [0]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'x 0 175px;\n}\n\nheader.header .logo {\n    margin-top'

Next Char Predictions: 
 'd]2P\teR~&BGenkLoCX*•>b;8Bt•n<EpTWnvawALr\n\ne?ytID{J'


Attach an optimizer, and a loss function
The standard tf.keras.losses.sparse_categorical_crossentropy loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the from_logits flag.

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (98, 50, 99)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.5953155


In [0]:
model.compile(optimizer='adam', loss=loss)

Configure checkpoints

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
684/684 [==============================] - 2290s 3s/step - loss: 1.3685
Epoch 2/10
414/684 [=================>............] - ETA: 14:14 - loss: 0.7530

In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
model.summary()

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.5

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string=u"<!DOCTYPE html> "))